# 4 텍스트와 바이트
- 유니코드 문자열과 이진 시퀀스, 그리고 이 둘 간의 변환에 사용되는 인코딩

## 4.1 문자 문제
- 문자열은 문자의 열이며, 여기서 문자는 유니코드 문자이다.
- 유니코드 표준은 문자의 단위 원소와 특정 바이트 표현을 명확히 구분한다.
  - 문자의 단위 원소 (코드 포인트)는 10진수 0부터 1,114,111까지의 숫자이며, 유니코드 표쥰에서는 'U+' 접두사를 붙여 4자리에서 6자리 사이의 16진수로 표현
- 코드 포인트를 바이트로 변환하는 것은 **인코딩**, 바이트를 코드 포인트로 변환하는 것을 **디코딩**이라고 한다.

In [1]:
# 예제 4-1
s = 'café'
len(s)  # 문자열은 네 개의 유니코드 문자를 갖고 있다.

4

In [2]:
b = s.encode('utf8')  # UTF-8 인코딩을 사용하여 str을 bytes로 인코딩
b, len(b)  # bytes가 다섯 바이트로 구성되어 있다. é가 UTF-8에서 두 바이트로 인코딩되기 때문이다.

(b'caf\xc3\xa9', 5)

In [3]:
b.decode('utf-8')

'café'

## 4.2 바이트에 대한 기본 지식
- 이진 시퀀스를 위한 내장 자료형으로는 `bytes` (불변형)와 `bytearray` (가변형)가 있다.
- 이 둘의 각 항목은 0과 255사이의 정수이다.
- 하지만 출력 리터럴은 아스키 코드 또는 16진수로 보여준다.


In [4]:
# 예제 4-2 bytes와 bytearray로 저장한 5바이트 시퀀스
cafe = bytes('café', encoding='utf_8')

fmt = '{0:>3d} : {1:>4s} : {2:>2s}'
print(cafe)  # b''의 b가 바이트를 의미
print('int :  hex : chr')
print('==================')
for item in cafe:
    print(fmt.format(item, hex(item), chr(item)))

b'caf\xc3\xa9'
int :  hex : chr
 99 : 0x63 :  c
 97 : 0x61 :  a
102 : 0x66 :  f
195 : 0xc3 :  Ã
169 : 0xa9 :  ©


In [5]:
cafe_arr = bytearray(cafe)
cafe_arr

bytearray(b'caf\xc3\xa9')

- `format()`, `format_map()`을 제외하고 `str`이 제공하는 메서드를 모두 제공
- 유니코드 데이터 관련 메서드 제공


In [6]:
# 16진수로부터 이진 시퀀스 만들기
b = bytes.fromhex('31 4B CE A9')
print(b)
b.decode('utf-8')


b'1K\xce\xa9'


'1KΩ'

- 버퍼 등의 객체로부터 이진 시퀀스를 생성하는 방법은 저수준 연산으로서, 형변환이 필요할 수 있다.


In [7]:
# 예제 4.3 배열의 원시 데이터에서 bytes 초기화하기
import array

numbers = array.array('h', [-2, -1, 0, 1, 2])  # h: short int (16 bit)
octets = bytes(numbers)
octets

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

### 4.2.1 구조체와 메모리 뷰
`struct` 모듈은 패킹된 바이트를 다양한 형의 필드로 구성된 튜플로 분석하고, 튜플을 패킹된 바이트로 변환하는 함수를 제공

In [8]:
# 예제 4-4 memoryview와 struct를 사용해서 GIF 이미지 헤더 조사하기
import struct
fmt = '<3s3sHH'

with open('filter.gif', 'rb') as fp:
    img = memoryview(fp.read())
    
header = img[:10]  # 새로운 memorhview 객체 반환 (바이트 10개 항목 주지 않음)
bytes(header)

b'GIF89a\x80\x02\x86\x01'

In [9]:
struct.unpack(fmt, header)  # (종류, 버전, 너비, 높이) 튜플로 언패킹

(b'GIF', b'89a', 640, 390)

In [10]:
del header
del img

## 4.3 기본 인코더/디코더
- 파이썬에서는 100여 개의 코덱(인코더/디코더)을 제공

In [11]:
# 예제 4-5 전혀 다른 바이트 시퀀스를 만드는 세 개의 코덱으로 인코딩한 'El Niño' 문자열
for codec in ['latin_1', 'utf_8', 'utf_16']:
    print(codec, 'El Niño'.encode(codec), sep='\t')

latin_1	b'El Ni\xf1o'
utf_8	b'El Ni\xc3\xb1o'
utf_16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


## 4.4 인코딩/디코딩 문제 이해하기
### 4.4.1 UnicodeEncodeError 처리하기
- 주어진 코덱에 문자가 대상 인코딩에 정의되어 있지 않으면 발생


In [12]:
# 예제 4-6 바이트로 인코딩하기: 성공 및 에러 처리
city = 'São Paulo'
city.encode('utf_8')

b'S\xc3\xa3o Paulo'

In [13]:
city.encode('utf_16')

b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'

In [14]:
city.encode('iso8859_1')

b'S\xe3o Paulo'

In [15]:
city.encode('cp437')

UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>

In [16]:
city.encode('cp437', errors='ignore')  # 스킵하고 진행

b'So Paulo'

In [17]:
city.encode('cp437', errors='replace')  # ?로 치환

b'S?o Paulo'

In [18]:
city.encode('cp437', errors='xmlcharrefreplace')  # XML개체로 치환

b'S&#227;o Paulo'

### 4,4,2 UnicodeDecodeError 처리하기
- 모든 바이트가 정당한 아스키 문자가 될 수 없다.
- 모든 바이트 시퀀스가 정당한 UTF-8이나 UTF-16 문자가 되는 것은 아니다.
- 이진 시퀀스를 텍스트로 변환할 때 정당한 문자로 변환할 수 없으면 발생


In [19]:
# 예제 4-7 bytes에서 str으로 디코딩하기: 성공 및 에러 처리
octets = b'Montr\xe9al'  # Montréal
octets.decode('cp1252')

'Montréal'

In [20]:
octets.decode('iso8859_7')  #  잘못됌

'Montrιal'

In [21]:
octets.decode('koi8_r')  #  잘못됌

'MontrИal'

In [22]:
octets.decode('utf-8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [23]:
octets.decode('utf-8', errors='replace')

'Montr�al'

### 4.4.3 예상과 달리 인코딩된 모듈을 로딩할 때 발생하는 SyntaxError
- UTF-8을 소스 코드 기본 인코딩 방식으로 사용

In [24]:
import ola

Olá, Mundo!


### 4.4.4 바이트 시퀀스의 인코딩 방식을 알아내는 방법
- 바이트 시퀀스가 주어졌을 때, 이 바이트 시퀀스의 인코딩 방식을 알아낼 수 없다.
- 통계적으로 추정할 수는 있다

### 4.4.5 BOM: 유용한 깨진 문자
  - 바이트 순서 표시 (byte order mark)

In [28]:
u16 = 'El Niño'.encode('utf_16')
u16  # E 앞에 2개의 16진수가 있는 것을 확인할 수 있다.

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'

- `\xff\xfe`이 BOM으로 인코딩한 인텔 CPU의 리틀엔디언 바이트 순서를 나타낸다.
- 리틀엔디언에서는 첫 2개로 촤하위 바이트 255, 254가 먼저 나온 후, 코드 포인트가 2개 바이트로 인코딩된다.
  - 255, 254의 의미는 ZERO WIDTH NO-BREAK SPACE (U+FEFF)이다.
- 엔디언 문제는 한 바이트 이상을 워드로 사용하는 UTF-16, UTF-32에만 영향을 준다.
  - UTF-8은 엔디언에 상관 없이 동일한 바이트 시퀀스를 생성한다.

In [26]:
list(u16)

[255, 254, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]

- 인코딩 방식에 le 또는 be를 명시하면 더 이상 BOM을 생성하지 않는다.


In [30]:
u16_le = 'El Niño'.encode('utf_16le')
list(u16_le)  # E 앞에 2개의 16진수가 있는 것을 확인할 수 있다.

[69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]

In [32]:
u16_be = 'El Niño'.encode('utf_16be')
list(u16_be)  # E 앞에 2개의 16진수가 있는 것을 확인할 수 있다.

[0, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111]

## 4.5 텍스트 파일 다루기
텍스트를 처리하는 최고의 방법
- 파일을 열 때 가능한 빨리 `bytes`를 `str`으로 변환 (디코딩)
- 파일 처리 중에는 오로지 `str` 객체로 다루고, 인코딩 또는 디코딩을 자제
- 파일을 출력할 때, `str`을 `bytes`로 인코딩

In [33]:
# 예제 4-9 플랫폼 인코딩 문제
open('cafe.txt', 'w', encoding='utf_8').write('café')

4

- 만약 encoding을 지정해주지 않으면, default로 인코딩한다. 기본 설정의 인코딩을 사용하는 것보다 인코딩을 지정해주는 편이 좋다.

In [34]:
open('cafe.txt').read()    

'café'

In [35]:
# 예제 4-10 버그와 해결 방법
import os

os.stat('cafe.txt').st_size

5

In [37]:
fp2 = open('cafe.txt')
print(fp2.encoding)
fp2.close()

UTF-8


In [38]:
fp4 = open('cafe.txt', 'rb')  # 이진 모드로 여는 것은 좋지 않다. 일반적으로 래스터 이미지 등 이진 파일을 열 때만 이진 모드를 사용한다.
print(fp4.read())
fp4.close()

b'caf\xc3\xa9'


## 4.6 제대로 비교하기 위해 유니코드 정규화하기
- 유니코드에는 결합 문자가 있기 때문에 문자열 비교가 간단하지 않다.
- 앞 문자에 연결되는 발음 구별 기호는 인쇄할 때 앞 문자와 하나로 결합되어 출력된다.
  

- 코드 포인트 U+0301 (`\u0301`)은 COMBINING ACUTE ACCENT
- 유니코드에서는 `é`와 `e\u0301`을 규범적으로 동일하다고 정의. 따라서 프로그램에서는 이 둘이 같아야 한다. 

In [39]:
s1 = 'café'
s2 = 'cafe\u0301'

print(s1, s2)

print(len(s1), len(s2))

print(s1 == s2)

café café
4 5
False


- 이 문제를 해결하기 위해 `unicodedata.normalize()` 함수가 제공하는 유니코드 정규화를 이용해야 한다.
- Normalize qkdqjqdpsms NFC, NFD, NFKC, NFKD가 있다.
  - NFC: 결합한 짧은 상태로 비교
  - NFD: 결합 해제한 긴 상태로 비교
- 사용자가 입력한 파일을 저장할 때, NFC로 normalize해주고 저장하는 것이 좋다. (W3C가 추천하는 정규화 형식)

In [40]:
from unicodedata import normalize
s1 = 'café'
s2 = 'cafe\u0301'

print(len(s1), len(s2))
print(len(normalize('NFC', s1)), len(normalize('NFC', s2)))
print(len(normalize('NFD', s1)), len(normalize('NFD', s2)))

print(normalize('NFC', s1) == normalize('NFC', s2))
print(normalize('NFD', s1) == normalize('NFD', s2))


4 5
4 4
5 5
True
True


- NFC에 의해 한 문자가 다른 문자로 정규화될 때도 있다.
- 똑같이 생긴 두 옴이 서로 다르다고 나오는 것을 방지하기 위해 normalize를 해줘야 한다.

In [44]:
from unicodedata import normalize, name

ohm = '\u2126'
print("name: ", name(ohm))

ohm_c = normalize('NFC', ohm)
print("name: ", name(ohm_c))

print(ohm == ohm_c)

print(normalize('NFC', ohm) == normalize('NFC', ohm_c))


name:  OHM SIGN
name:  GREEK CAPITAL LETTER OMEGA
False
True


- 정규화 방식에서 K는 호환성을 나타낸다. 정규화를 넘어 호환성 문자에 영향을 미친다.
- 유니코드 표준은 하나의 문주에 하나의 코드만을 갖는 것이 원칙이었지만, 다른 인코딩 방식과의 호환성을 위해 한 문자에 두 코드를 갖는 경우가 있다.
- 포매팅 손실이 발생하더라도 선호하는 형태의 하나이상의 문자로 구성된 호환성 분할로 치환한다.
  - 데이터 치환시 발생하는 손실 때문에 영구 저장할 때 사용을 자제 해야 한다.

In [77]:
from unicodedata import normalize, name

half = '\u00bd'
print(half)
print(normalize('NFKC', half))

print()

four_squared = '4²'
print(four_squared)
print(normalize('NFKC', four_squared))

print()

micro = '\u00b5'  # 마이크로
print(micro)  

micro_kc = normalize('NFKC', micro)  # 소문자 뮤
print(micro_kc)

print(ord(micro), ord(micro_kc))

print(name(micro), '|', name(micro_kc))

print('µ'  == 'μ')

½
1⁄2

4²
42

µ
μ
181 956
MICRO SIGN | GREEK SMALL LETTER MU
False


### 4.6.1 케이스 폴딩
- 모든 텍스트를 소문자로 변환하는 연산
- `str.lower()`와 다른 코드 포인트가 116개 (0.11%)에 해당
  - latin1만 담고 있는 문자열의 경우 `str.lower()`와 동일

In [80]:
micro = '\u00b5'
print(name(micro))

micro_cf = micro.casefold()
print(name(micro_cf))

eszett = 'ß'
print(name(eszett))

eszett_cf = eszett.casefold()
print(eszett_cf)

MICRO SIGN
GREEK SMALL LETTER MU
LATIN SMALL LETTER SHARP S
ss


### 4.6.2 정규화된 텍스트 매칭을 위한 유틸리티 함수
- NFC 방식은 대부분의 애플리케이션에서 사용할 수 있는 최고의 정규화된 형태
- `str`, `casefold`는 대소문자 구분 없이 문자를 비교할 때 좋음

In [84]:
# 예제 4-13 normeq.py: 정규화된 유니코드 문자열 비교
from unicodedata import normalize

def nfc_equal(str1, str2):
    return normalize('NFC', str1) == normalize('NFC', str2)


def fold_equal(str1, str2):
    return (normalize('NFC', str1).casefold() == 
            normalize('NFC', str2).casefold())
    
s1 = 'café'
s2 = 'cafe\u0301'
print(s1 == s2)
print(nfc_equal(s1, s2))
print(nfc_equal('A', 'a'))

print()

s3 = 'Straße'
s4 = 'strasse'

print(s3 == s4)
print(nfc_equal(s3, s4))
print(fold_equal(s3, s4))
print(fold_equal(s1, s2))
print(fold_equal('A', 'a'))


False
True
False

False
False
True
True
True


### 4.6.3 극단적인 '정규화': 발음 구별 기호 제거하기
- URL에 이스케이프 처리가 있을 때 비교하기 좋음

In [86]:
# 예제 4-14 결합 표시를 모두 제거하는 함수
import unicodedata
import string

def shave_marks(txt):
    norm_txt = unicodedata.normalize('NFD', txt)  # 결합 분해
    shaved = ''.join(c for c in norm_txt if not unicodedata.combining(c)) # 결합표시 제거
    return unicodedata.normalize('NFC', shaved)  # 문자 재결합

order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
print(order)
print(shave_marks(order))

print()

greek = 'Ζέφυρος, Zéfiro'
print(greek)
print(shave_marks(greek))

“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”
“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”

Ζέφυρος, Zéfiro
Ζεφυρος, Zefiro


In [87]:
# 예제 4-14 결합 표시를 모두 제거하는 함수
import unicodedata
import string

def shave_marks_latin(txt):
    """Remove all diacritic marks from Latin base characters"""
    norm_txt = unicodedata.normalize('NFD', txt)  # 결합분해
    latin_base = False
    keepers = []
    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:   # 결합문자일시
            continue  # 추가하지 않음
        keepers.append(c)
        # 새로운 기반 문자를 찾아내고 라틴 문자인지 확인
        if not unicodedata.combining(c):
            latin_base = c in string.ascii_letters
    shaved = ''.join(keepers)
    return unicodedata.normalize('NFC', shaved)

order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
print(order)
print(shave_marks_latin(order))

print()

greek = 'Ζέφυρος, Zéfiro'
print(greek)
print(shave_marks_latin(greek))

“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”
“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”

Ζέφυρος, Zéfiro
Ζέφυρος, Zefiro


In [88]:
# 예제 4-17 서양 활자 기호를 아시크로 변환하기

single_map = str.maketrans("""‚ƒ„†ˆ‹‘’“”•–—˜›""",  # 문자 대 문자 치환 매핑 테이블
                           """'f"*^<''""---~>""")

multi_map = str.maketrans({  # 문자 대 문자열 치환 매핑 테이블
    '€': '<euro>',
    '…': '...',
    'Œ': 'OE',
    '™': '(TM)',
    'œ': 'oe',
    '‰': '<per mille>',
    '‡': '**',
})

multi_map.update(single_map)  # 두 개 매핑 테이블 병합


def dewinize(txt):
    """Replace Win1252 symbols with ASCII chars or sequences"""
    return txt.translate(multi_map)  # <4>


def asciize(txt):
    no_marks = shave_marks_latin(dewinize(txt))     # <5>
    no_marks = no_marks.replace('ß', 'ss')          # <6>
    return unicodedata.normalize('NFKC', no_marks)  # <7>

order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
print(order)
print(dewinize(order))
print(asciize(order))


“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”
"Herr Voß: - ½ cup of OEtker(TM) caffè latte - bowl of açaí."
"Herr Voss: - 1⁄2 cup of OEtker(TM) caffe latte - bowl of acai."


## 4.7 유니코드 텍스트 정렬하기
- 문자열의 경우에는 각 단어의 코드 포인트를 비교하여 정렬
  - 비아스키 문자를 사용하는 경우 코드 포인트 기준 정렬이 부적절할 수 있다.

In [7]:
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(fruits)

['acerola', 'atemoia', 'açaí', 'caju', 'cajá']

- `locale.strxfrm()`은 현지어 비교에 사용할 수 있는 문자열로 변환

In [8]:
# locale.strxfrm() 함수를 정렬키로 사용하기
import locale

locale.setlocale(locale.LC_COLLATE, 'pt_BR.UTF-8')
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted_fruits = sorted(fruits, key=locale.strxfrm)
sorted_fruits

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']

In [24]:
list(map(locale.strxfrm, fruits))

['\x0e\n\x0e\x02\x0e#\x0e\x9f\x01\x01\x01\x01',
 '\x0e\x02\x0e\x99\x0e!\x0eQ\x0e|\x0e2\x0e\x02\x01\x01\x01\x01',
 '\x0e\n\x0e\x02\x0e5\x0e\x02\x01\x02\x02\x02\x0e\x01\x01\x01',
 '\x0e\x02\x0e\n\x0e\x02\x0e2\x01\x02\x1c\x02\x0e\x01\x01\x01',
 '\x0e\x02\x0e\n\x0e!\x0e\x8a\x0e|\x0eH\x0e\x02\x01\x01\x01\x01']

### 4.7.1 유니코드 대조 알고리즘을 이용한 정렬

In [9]:
# 예제 4-20 pyuca.Collator.sort_key() 메서드 사용하기
import pyuca
coll = pyuca.Collator()
fruits = ['cafu', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted_fruits = sorted(fruits, key=coll.sort_key)
sorted_fruits

['açaí', 'acerola', 'atemoia', 'cafu', 'cajá']

In [16]:
list(map(coll.sort_key, fruits))

[(7290, 7239, 7397, 7861, 0, 32, 32, 32, 32, 0, 2, 2, 2, 2, 0),
 (7239,
  7829,
  7338,
  7594,
  7645,
  7474,
  7239,
  0,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0),
 (7290, 7239, 7500, 7239, 0, 32, 32, 32, 32, 36, 0, 2, 2, 2, 2, 2, 0),
 (7239, 7290, 7239, 7474, 0, 32, 32, 48, 32, 32, 36, 0, 2, 2, 2, 2, 2, 2, 0),
 (7239,
  7290,
  7338,
  7731,
  7645,
  7543,
  7239,
  0,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0)]

## 4.8 유니코드 데이터베이스
- 유니코드 표준에서 텍스트 파일의 형태로 모든 정보를 담은 데이터 베이스를 제공
  - 코드 포인트 <-> 문자명 테이블
  - 문자에 대한 메타데이터: 문자 출력 가능 여부, 문자 여부, 십진수 여부, 수치형 여부
  - 각 문자의 연관 방법
- `str`의 여러 메서드가 이 데이터 베이스를 사용

In [32]:
# 예제 4-21 유니코드 데이터베이스 수치형 문자 메타데이터 사용 예
import unicodedata
import re

re_digit = re.compile(r'\d')

sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
    print(
        'U+%04x' % ord(char),                          # 코드 포인트
        char.center(6),                                # 길이가 6인 str의 중앙에 놓인 문자
        're_dig' if re_digit.match(char) else '-',     # 정규 표현식과 일치하면 re_dig
        'isdig' if char.isdigit() else '-',            # 숫자: 단일문자가 숫자 모양으로 생기면 
        'isnum' if char.isnumeric() else '-',          # 수치형: 숫자값 표현을 위한 문자열
        format(unicodedata.numeric(char), '5.2f'),     # 나타내는 숫자
        unicodedata.name(char),                        # 유니코드 문자명
        sep='\t'
    )

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


## 4.9 이중 모드 str 및 bytes API
### 4.9.1 정규 표현식에서의 str과 bytes
- bytes에 정규표현식을 사용할 경우, 아스키 범위를 벗어나는 문자들은 숫자나 단어로 처리하지 않는다.

In [34]:
# 예제 4-22 ramanujan.py 간단한 str과 bytes 정규 표현식의 동장 비교
import re

re_numbers_str = re.compile(r'\d+')     # 숫자
re_words_str = re.compile(r'\w+')       # 
re_numbers_bytes = re.compile(rb'\d+')  # <2>
re_words_bytes = re.compile(rb'\w+')

text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef" 
            " as 1729 = 1³ + 12³ = 9³ + 10³.")        

text_bytes = text_str.encode('utf_8')  # <5>

print('Text', repr(text_str), sep='\n  ')
print('Numbers')
print('  str  :', re_numbers_str.findall(text_str))     
print('  bytes:', re_numbers_bytes.findall(text_bytes)) 

print('Words')
print('  str  :', re_words_str.findall(text_str))      
print('  bytes:', re_words_bytes.findall(text_bytes))  

Text
  'Ramanujan saw ௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
  str  : ['௧௭௨௯', '1729', '1', '12', '9', '10']
  bytes: [b'1729', b'1', b'12', b'9', b'10']
Words
  str  : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
  bytes: [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


### 4.9.2 os 모듈에서 str과 bytes
- OS에서의 파일명들은 이떠한 인코딩 체계에서도 올바르지 않은 바이트 시퀀스로 구성되어 있다.
- 

In [39]:
import os

print(os.listdir('.'))
print(os.listdir(b'.'))

['.ipynb_checkpoints', 'cafe.txt', 'Chapter4.ipynb', 'digits-of-π.txt', 'filter.gif', 'ola.py', '__pycache__']
[b'.ipynb_checkpoints', b'cafe.txt', b'Chapter4.ipynb', b'digits-of-\xcf\x80.txt', b'filter.gif', b'ola.py', b'__pycache__']
